In [20]:
import os
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import func4behav as fb
from scipy.stats import lognorm, exponnorm, invgauss
from scipy.optimize import curve_fit
from sklearn.metrics import mean_squared_error
from pyddm import Model, Sample, Fittable, Fitted
from pyddm.models import DriftConstant, NoiseConstant, BoundConstant, OverlayNonDecision, ICPointSourceCenter, LossRobustLikelihood
from pyddm.functions import fit_adjust_model, display_model
import pyddm.plot

import imp
imp.reload(fb)

<module 'func4behav' from 'c:\\Users\\Tianyi Zheng\\OneDrive - neuron.t.u-tokyo.ac.jp\\Documents\\zheng\\mywork\\attention_tES\\tes-attention\\processing\\behavior\\func4behav.py'>

In [21]:
def distribution_param(case_list, cutoff):

    _, expt_info = fb.create_allsubs_compare()
    param_record = pd.DataFrame(columns=['subject id', 'case', 'group', 'session', 'method', 'params', 'log likelihood', 'AIC'])
    
    session_name = ['before', 'after']
    methods_name = ['shift lognorm', 'exgaussian', 'ddm', 'shift invgauss']
    methods = [fb.fit_shift_lognorm, fb.fit_exgaussian, fb.fit_ddm, fb.fit_shift_invgauss]

    # Loops
    record_idx = 0
    for subject_id in range(1,19):
        print("Subject id:" + str(subject_id) + " is processing...")
        real_or_sham = expt_info.loc[expt_info['subject id'] == subject_id, 'Real stimulation'].values[0]

        for case in case_list:
            print("Case:" + str(case) + " is processing...")
            rt_before, rt_after = make_rt_data(subject_id, case, cutoff)

            for method_idx, method in enumerate(methods):
                for session_idx, rt in enumerate([rt_before, rt_after]):
                    param, log_L, AIC = method(rt, cutoff)

                    param_record.loc[record_idx, 'subject id'] = subject_id
                    param_record.loc[record_idx, 'case'] = case
                    param_record.loc[record_idx, 'group'] = "real" if real_or_sham == 1 else "sham"
                    param_record.loc[record_idx, 'session'] = session_name[session_idx]
                    param_record.loc[record_idx, 'method'] = methods_name[method_idx]
                    param_record.loc[record_idx, 'params'] = param
                    param_record.loc[record_idx, 'log likelihood'] = log_L
                    param_record.loc[record_idx, 'AIC'] = AIC

                    record_idx += 1
                    
    return param_record


def make_rt_data(subject_id, case, cutoff):
    
    behavior_before, behavior_after = fb.load_behavior(subject_id)
    behavior_before, behavior_after = fb.filter_behav(case, behavior_before, behavior_after)
    rt_before = behavior_before.loc[(behavior_before['response'] == 1) & (behavior_before['reaction time'] > 0.05) & (behavior_before['reaction time'] < cutoff)]
    rt_after = behavior_after.loc[(behavior_after['response'] == 1) & (behavior_after['reaction time'] > 0.05) & (behavior_after['reaction time'] < cutoff)]

    rt_before = rt_before['reaction time'].values
    rt_after = rt_after['reaction time'].values

    return rt_before, rt_after

In [22]:
all_real = [1, 3, 4, 5, 9, 12, 13, 17, 18]
all_sham = [2, 6, 7, 8, 10, 11, 14, 15, 16]
case_list = ['all', 'endo', 'exo', 'valid', 'endo valid', 'exo valid', 'invalid', 'endo invalid', 'exo invalid', 'stim left', 'endo stim left', 'endo stim right', 'stim right', 'exo stim left', 'exo stim right']
cutoff = 1
param_record = distribution_param(case_list, cutoff)
param_record.to_csv('../../../docs/report/data/param_record.csv', index=False)

Subject id:1 is processing...
Case:all is processing...
[6.722364450855462, 1.2159748160439794, 0.8122197842759846, 0.1112698182135084]
[9.652520824924204, 2.095797061118738, 1.3836681041017924, 0.050784276157332456]
Case:endo is processing...
[7.483740154619388, 1.6099985298353157, 1.0663082187181374, 0.10625496391553191]
[6.500674679344469, 1.0231456516266813, 0.989184486233535, 0.05056581074418032]
Case:exo is processing...
[9.730064579826468, 1.1984586488969258, 1.059434510301042, 0.1068317532280095]
[9.805758228707331, 2.4213932195318484, 1.3175842134184053, 0.05052669002835092]
Case:valid is processing...
[8.657223019757694, 1.38003581545366, 0.9803505549808247, 0.1082986006395125]
[8.652812288775474, 1.7876654814198558, 1.1443581504873934, 0.05081695109948163]
Case:endo valid is processing...
[5.295673860931959, 0.9556080674097048, 0.6856794321416159, 0.10234755691211042]
[7.976580590370359, 1.0324246812388604, 1.0714235629031836, 0.0516716490078597]
Case:exo valid is processing

## get P value

In [23]:
param_record_path = os.path.join('..', '..', '..', 'docs', 'report', 'data', 'param_record.csv')
param_record = pd.read_csv(param_record_path)
param_record

,subject id,case,group,session,method,params,log likelihood,AIC
0,1,all,real,before,shift lognorm,[0.53045063 0.09696554 0.11904519],184.279441,-362.558881
1,1,all,real,after,shift lognorm,[ 0.17276475 0.34512215 -0.15826391],162.489464,-318.978928
2,1,all,real,before,exgaussian,"[2.9564068064090367, 0.1736784921094102, 0.019...",185.574583,-365.149166
3,1,all,real,after,exgaussian,"[1.1293728835241597, 0.1461682210666338, 0.040...",164.208207,-322.416415
4,1,all,real,before,ddm,"[6.722364450855462, 1.2159748160439794, 0.8122...",182.517591,-357.035182
...,...,...,...,...,...,...,...,...
2155,18,exo stim right,real,after,exgaussian,"[4.223375125196021, 0.24303794121277486, 0.014...",48.557894,-91.115789
2156,18,exo stim right,real,before,ddm,"[9.400482379446196, 1.5093073559137486, 2.1868...",36.627674,-65.255347
2157,18,exo stim right,real,after,ddm,"[7.72099553032849, 1.4410927034382375, 0.77699...",48.680689,-89.361378
2158,18,exo stim right,real,before,shift invgauss,"[0.11336152603077676, 0.10825551392430313, 2.0...",36.628786,-67.257573
